**Introduction**
As part of this challenge we are trying to predict sales of various items sold by Favorita retailer. Following are the datasets provided to us:
*  train.csv
* stores.csv
* transactions.csv
* items.csv
* holidays_events.csv
* oils.csv

**Description of Dataset**
Please find link to dataset explaination here[https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data](http://www.kaggle.com/c/favorita-grocery-sales-forecasting/data)


* The first step is to read the data from various datasets and carry out some basic analysis. We use pandas read_csv to read the data set.
* The second step is to analyse the dataset by means of various graphs and try to understand /co-relate with different datasets we are given. 
* The third step is to carry out feature engineering whereby we identify some of the key features from the dataset.
* The fourth and final step is to train the model and test it. May be we could do K Means??

**I. Reading various data from input**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import xgboost as xgb

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
#Since training data is huge, so I am planning to read few millions of rows from the csv file.
train = pd.read_csv('../input/train.csv', usecols=[0, 1, 2, 3, 4,5], parse_dates=["date"],skiprows=range(1, 89940990))

#print the last 10 rows of the data, this will help us to think what we can dow with the data.
train.tail(5)

Training data and items csv are some way or other way related to each other. So I think lets read the items csv and try to merge with training data which will help us in getting more insight from the data.

In [ ]:
items = pd.read_csv("../input/items.csv")

In [ ]:
train_items = pd.merge(train, items, how='inner')
train_items.tail(5)

After merging two sets of data (training and items) we can now carry out analysis of items sold.

In [ ]:
#Lets find out most popular item ordered by people across the 6 millions rows we have read.
#We will group by item_nbr and add the unit sales.
df = train_items['unit_sales'].groupby(train_items['item_nbr']).sum()
#In order to find top 10 popular items we will sort the numpy array and pick the top 10 from
#the list.
df = df.sort_values()
df_highest = df.nlargest(n=10)

#Plot the highest list of items.
df_highest.plot(kind='bar',figsize = (10,10),  title = "Top 10 items sold across all stores")
plt.show()

In [ ]:
#Next we find lowest/less demand product. We use nsmallest to find the bottom 10 items,
#probably it doesn;t matter even if we stock them.
df_lowest = df.nsmallest(n=10)
df_lowest.plot(kind='bar',figsize = (10,10),  title = "Bottom 10 items sold")
plt.show()

In [ ]:
#Next we could find out popular items in a given year. This will be useful to find out 
#if there were any new items introduced in the recent times.
#In order to do that we need to covert the date field into python date format and then
# extract various fields from it.

train_items['date'] = pd.to_datetime(train_items['date'], format='%Y-%m-%d')
train_items['day_item_purchased'] = train_items['date'].dt.day
train_items['month_item_purchased'] =train_items['date'].dt.month
train_items['quarter_item_purchased'] = train_items['date'].dt.quarter
train_items['year_item_purchased'] = train_items['date'].dt.year

In [ ]:
train_items.drop('date', axis=1, inplace=True)

In [ ]:
#Lets print out new training dataset
print (train_items.tail(2))

In [ ]:
df_year = train_items.groupby(['quarter_item_purchased', 'item_nbr'])['unit_sales'].sum()
df_year = df_year.sort_values()
df_year_highest = df_year.nlargest(n=10)
#Plot the highest list of items.
df_year_highest.plot(kind='bar',figsize = (10,10),  title = "Top items sold Quarterly")
plt.show()

In [ ]:
plt.figure(figsize=(9,10))
df_items = train_items.groupby(['family'])['unit_sales'].sum()
df_items = df_items.sort_values()
df_items_highest = df_items.nlargest(n=10)
plt.pie(df_items_highest, labels=df_items_highest.index,shadow=False,autopct='%1.1f%%')
plt.tight_layout()
plt.show()


In [ ]:
grocery_info = train_items.loc[train_items['family'] == 'GROCERY I']

In [ ]:
plt.figure(figsize=(12,12))
#print (grocery_info.tail(2))
plt.plot(grocery_info['day_item_purchased'],grocery_info['unit_sales'])
plt.show()

In [ ]:
plt.figure(figsize=(9,10))
df_items = train_items.groupby(['family','perishable'])['unit_sales'].sum()
df_items = df_items.sort_values()
df_items_perish_highest = df_items.nlargest(n=10)
plt.pie(df_items_perish_highest, labels=df_items_perish_highest.index,shadow=False,autopct='%1.1f%%')
plt.tight_layout()
plt.show()

Lets read the transactions data and carry out analysis.

In [ ]:
transaction = pd.read_csv("../input/transactions.csv")

Convert date to pandas data time format, so that  we can group items for a given time frame (monthly, yearly, quarterly)

In [ ]:
transaction['date'] = pd.to_datetime(transaction['date'], format='%Y-%m-%d')
transaction['day_item_purchased'] = transaction['date'].dt.day
transaction['month_item_purchased'] =transaction['date'].dt.month
transaction['quarter_item_purchased'] = transaction['date'].dt.quarter
transaction['year_item_purchased'] = transaction['date'].dt.year
print (transaction.tail(2))

In [ ]:
plt.figure(figsize=(25,25))
plt.plot(transaction['date'],transaction['transactions'])
plt.show()


In [ ]:
plt.figure(figsize=(8,12))
trans_day = transaction['transactions'].groupby(transaction['year_item_purchased']).sum()
trans_day.plot(kind='bar')
plt.show()

In [ ]:
stores = pd.read_csv("../input/stores.csv")
print (stores.head())

In [ ]:
#Lets find out number of cities in each state, which in nothing but finding out number of stores in each
#in each state.
df = stores['city'].groupby(stores['state']).count()
df.plot(kind='bar', figsize = (12,8), yticks=np.arange(min(df), max(df)+1, 1.0), title = "Number of cities in each state")
plt.show()

In [ ]:
#Looks like onpromotion field is always NaN, if so we will get rid of that columns 
#from the training data
print(train['onpromotion'].notnull().any())
train_new=train.drop('onpromotion',axis=1)
print(train_new.tail(5))

In [ ]:
oils = pd.read_csv("../input/oil.csv")
oils['date'] = pd.to_datetime(oils['date'], format='%Y-%m-%d')
oils['day_item_purchased'] = oils['date'].dt.day
oils['month_item_purchased'] =oils['date'].dt.month
oils['quarter_item_purchased'] = oils['date'].dt.quarter
oils['year_item_purchased'] = oils['date'].dt.year

In [ ]:
plt.figure(figsize=(25,25))
#trans_day = transaction['transactions'].groupby(transaction['year_item_purchased']).sum()
plt.plot(oils['date'],oils['dcoilwtico'])
#trans_day.plot(kind='bar')
plt.show()

In [ ]:
print(train_items.tail(2))
#train_items.fillna(1,inplace=True)
train_items.loc[(train_items.unit_sales<0),'unit_sales'] = 0 

In [ ]:
train_items['family'] = train_items['family'].astype('category')
train_items['onpromotion'] = train_items['onpromotion'].astype('category')
train_items['perishable'] = train_items['perishable'].astype('category')
cat_columns = train_items.select_dtypes(['category']).columns
train_items[cat_columns] = train_items[cat_columns].apply(lambda x: x.cat.codes)
print (train_items.tail(2))

In [ ]:
from sklearn.cross_validation import train_test_split
import xgboost as xgb
X_train, X_valid = train_test_split(train_items, test_size=0.20, random_state=20)
y_train = np.log1p(X_train.unit_sales)
y_valid = np.log1p(X_valid.unit_sales)
features = list(train_items.columns.values)
dtrain = xgb.DMatrix(X_train[features], y_train)
dvalid = xgb.DMatrix(X_valid[features], y_valid)

In [ ]:
def rmspe(y, yhat):
    return np.sqrt(np.mean((yhat/y-1) ** 2))

In [ ]:
def rmspe_xg(yhat, y):
    y = np.expm1(y.get_label())
    yhat = np.expm1(yhat)
    return "rmspe", rmspe(y,yhat)

In [ ]:
params = {"objective": "reg:linear",
          "booster" : "gbtree",
          "eta": 0.3,
          "max_depth": 10,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "silent": 1,
          "seed": 1
          }
num_boost_round=100

watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
gbm = xgb.train(params, dtrain, num_boost_round=num_boost_round, evals=watchlist, \
  early_stopping_rounds=300, verbose_eval=True)

print("Validating")
yhat = gbm.predict(xgb.DMatrix(X_valid[features]))
error = rmspe(X_valid.unit_sales.values, np.expm1(yhat))
print (error)
print('RMSPE: {:.6f}'.format(error))